In [1]:
# ============================
# STEP 1: Install dependencies
# ============================
!pip install -q langchain langgraph langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 10.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.7.0 which is incompatible.


In [2]:
# ============================
# STEP 2: Imports
# ============================
import os, json, textwrap
from typing import TypedDict

from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END

In [3]:
# ============================
# STEP 3: API key (Colab)
# ============================
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

In [4]:
# ============================
# STEP 4: LLM
# ============================
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.4)

In [5]:
# ============================
# STEP 5: Prompts
# ============================
extract_prompt = PromptTemplate(
    input_variables=["inquiry"],
    template=textwrap.dedent("""
    You are an intake assistant. Extract structured fields from the raw client inquiry.
    Return STRICT JSON with keys: name, company, email, topic, intent(one of: billing, support, sales, other),
    urgency(low/medium/high), needs_meeting(true/false), summary(20-40 words).

    Inquiry:
    {inquiry}
    JSON:
    """).strip()
)

route_prompt = PromptTemplate(
    input_variables=["extracted_json"],
    template=textwrap.dedent("""
    You are the orchestrator. Read this extracted info (JSON) and confirm the best route:
    - billing
    - support
    - sales
    - other
    Also return a 1-sentence rationale.

    Extracted:
    {extracted_json}

    Return STRICT JSON: {{ "route": "...", "rationale": "..."}}
    """).strip()
)

# Task-specific agent prompts
billing_agent_prompt = PromptTemplate(
    input_variables=["extracted_json"],
    template=textwrap.dedent("""
    You are a Billing Agent. Draft a clear, polite reply for the client based on:
    {extracted_json}

    Goals:
    - Address billing topic precisely.
    - If info is missing, ask 2 concise follow-up questions.
    - Offer next step.
    Limit to ~140 words.

    Reply:
    """).strip()
)

support_agent_prompt = PromptTemplate(
    input_variables=["extracted_json"],
    template=textwrap.dedent("""
    You are a Support Agent. Draft a step-by-step troubleshooting reply based on:
    {extracted_json}

    Goals:
    - Acknowledge issue.
    - Give 3-5 specific steps.
    - Ask for logs/screenshots if useful.
    - Offer escalation if unresolved.
    Limit to ~160 words.

    Reply:
    """).strip()
)

sales_agent_prompt = PromptTemplate(
    input_variables=["extracted_json"],
    template=textwrap.dedent("""
    You are a Sales Agent. Draft a consultative reply based on:
    {extracted_json}

    Goals:
    - Clarify needs.
    - Share 2-3 relevant value points.
    - Include 2 qualifying questions.
    - Offer a call if they want.
    Limit to ~140 words.

    Reply:
    """).strip()
)

schedule_prompt = PromptTemplate(
    input_variables=["extracted_json"],
    template=textwrap.dedent("""
    You are a scheduling helper. Based on timezone hints (if any) in:
    {extracted_json}
    Propose 3 short meeting time options for the next 3 business days (30-min slots), in ISO-like text.
    If "needs_meeting" false, return "No meeting required."
    """).strip()
)

evaluator_prompt = PromptTemplate(
    input_variables=["inquiry","reply"],
    template=textwrap.dedent("""
    Evaluate the drafted reply to the inquiry.

    Rules to pass:
    - Polite greeting & clear close
    - Directly addresses the main request
    - ≤ 180 words
    - Actionable next step or question

    Return STRICT JSON: {{"pass": true/false, "feedback": "1-3 concrete improvements"}}

    Inquiry:
    {inquiry}

    Draft reply:
    {reply}

    JSON:
    """).strip()
)

optimizer_prompt = PromptTemplate(
    input_variables=["inquiry","reply","feedback"],
    template=textwrap.dedent("""
    Improve the reply using the evaluator feedback. Keep it concise, polite, and ≤ 180 words.

    Inquiry:
    {inquiry}

    Current reply:
    {reply}

    Feedback to apply:
    {feedback}

    Return ONLY the improved reply text:
    """).strip()
)

crm_prompt = PromptTemplate(
    input_variables=["extracted_json","route","final_reply","schedule"],
    template=textwrap.dedent("""
    Create a compact CRM log entry as STRICT JSON with keys:
    contact_name, email, company, route, summary, next_action, meeting, reply_snippet

    Use:
    Extracted: {extracted_json}
    Route: {route}
    FinalReply: {final_reply}
    Schedule: {schedule}

    JSON:
    """).strip()
)

In [6]:
# ============================
# STEP 6: Graph State
# ============================
class State(TypedDict):
    raw_inquiry: str
    extracted: str         # JSON string
    route: str
    draft_reply: str
    evaluated: str         # JSON string
    improved_reply: str
    final_reply: str
    schedule: str
    crm_record: str

graph = StateGraph(State)

In [7]:
# ============================
# STEP 7: Nodes
# ============================
def intake_extract(state: State):
    r = llm.invoke(extract_prompt.format(inquiry=state["raw_inquiry"]))
    return {"extracted": r.content}

def triage_route(state: State):
    r = llm.invoke(route_prompt.format(extracted_json=state["extracted"]))
    # light parse to pull "route"; fall back to 'other'
    route = "other"
    try:
        route = json.loads(r.content).get("route","other").lower()
    except Exception:
        # naive fallback
        for key in ["billing","support","sales","other"]:
            if key in r.content.lower():
                route = key; break
    return {"route": route}

def agent_router(state: State):
    ex = state["extracted"]
    route = state["route"]
    if route == "billing":
        r = llm.invoke(billing_agent_prompt.format(extracted_json=ex))
    elif route == "support":
        r = llm.invoke(support_agent_prompt.format(extracted_json=ex))
    elif route == "sales":
        r = llm.invoke(sales_agent_prompt.format(extracted_json=ex))
    else:
        # generic polite reply
        generic = PromptTemplate(
            input_variables=["extracted_json"],
            template="Draft a polite, helpful reply and ask 2 clarifying questions based on: {extracted_json}\nLimit to ~140 words."
        )
        r = llm.invoke(generic.format(extracted_json=ex))
    return {"draft_reply": r.content}

def maybe_schedule(state: State):
    r = llm.invoke(schedule_prompt.format(extracted_json=state["extracted"]))
    return {"schedule": r.content.strip()}

def evaluate(state: State):
    r = llm.invoke(evaluator_prompt.format(inquiry=state["raw_inquiry"], reply=state["draft_reply"]))
    return {"evaluated": r.content}

def optimize_once(state: State):
    # default: keep original
    fb = ""
    try:
        fb = json.loads(state["evaluated"])["feedback"]
        # only optimize if "pass": false
        passed = json.loads(state["evaluated"]).get("pass", False)
        if passed:
            return {"improved_reply": state["draft_reply"], "final_reply": state["draft_reply"]}
    except Exception:
        # fallback to simplistic behavior
        if "pass" in state["evaluated"].lower() and "true" in state["evaluated"].lower():
            return {"improved_reply": state["draft_reply"], "final_reply": state["draft_reply"]}
        fb = "Tighten the reply, add a clear next step, and keep under 180 words."

    r = llm.invoke(
        optimizer_prompt.format(
            inquiry=state["raw_inquiry"],
            reply=state["draft_reply"],
            feedback=fb
        )
    )
    improved = r.content.strip()
    return {"improved_reply": improved, "final_reply": improved}

def build_crm(state: State):
    r = llm.invoke(
        crm_prompt.format(
            extracted_json=state["extracted"],
            route=state["route"],
            final_reply=state["final_reply"],
            schedule=state["schedule"]
        )
    )
    return {"crm_record": r.content}

In [8]:
# ============================
# STEP 8: Wire the graph
# ============================
graph.add_node("intake_extract", intake_extract)
graph.add_node("triage_route", triage_route)
graph.add_node("agent_router", agent_router)
graph.add_node("maybe_schedule", maybe_schedule)
graph.add_node("evaluate", evaluate)
graph.add_node("optimize_once", optimize_once)
graph.add_node("build_crm", build_crm)

graph.set_entry_point("intake_extract")
graph.add_edge("intake_extract", "triage_route")
graph.add_edge("triage_route", "agent_router")
graph.add_edge("agent_router", "maybe_schedule")
graph.add_edge("maybe_schedule", "evaluate")
graph.add_edge("evaluate", "optimize_once")
graph.add_edge("optimize_once", "build_crm")
graph.add_edge("build_crm", END)

app = graph.compile()

In [9]:
# ============================
# STEP 9: Run a demo
# ============================
raw_inquiry = """
Hi team,
I'm Priya from Stellar Labs. Our invoice #7843 seems to include two extra user seats
we didn't add. Can you check and correct it? If needed I'm happy to jump on a call
this week (IST afternoons). Please send the corrected amount and next steps.
Thanks! priya@stellarlabs.io
"""

result = app.invoke({"raw_inquiry": raw_inquiry})

print("=== FINAL REPLY TO CLIENT ===\n")
print(result["final_reply"].strip(), "\n")
print("=== SUGGESTED SCHEDULE ===\n")
print(result["schedule"].strip(), "\n")
print("=== CRM RECORD (JSON-ish) ===\n")
print(result["crm_record"].strip())

=== FINAL REPLY TO CLIENT ===

Subject: Re: Incorrect charge on invoice #7843

Hi Priya,

Thank you for reaching out about the charge for two extra user seats on invoice #7843. I'm sorry for any confusion and am happy to help investigate this for you.

To resolve this quickly, could you please help with two details?
1.  What are the names or email addresses associated with the extra seats?
2.  Can you confirm if anyone else on your team has permission to add users?

I see you’re available for a call. Please feel free to book a time that works for you on my calendar: [Calendar Link].

We’ll get this sorted out for you.

Best regards,

[Your Name]
Billing Agent 

=== SUGGESTED SCHEDULE ===

As there are no timezone hints, the following times are proposed in UTC.

Here are 3 options for a 30-minute meeting:
1.  `2024-07-16T14:00Z`
2.  `2024-07-17T10:30Z`
3.  `2024-07-18T15:00Z` 

=== CRM RECORD (JSON-ish) ===

```json
{
  "contact_name": "Priya",
  "email": "priya@stellarlabs.io",
  "comp